In [1]:
import numpy as np
import pandas as pd
from re import sub
from time import time

from comet_ml import Experiment

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras_tqdm import TQDMNotebookCallback as ktqdm
from keras.utils import normalize
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import TensorBoard
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from keras.initializers import RandomUniform, RandomNormal
from keras.layers.advanced_activations import LeakyReLU

from tensorflow.nn import relu, softmax

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, Normalizer
from sklearn.impute import SimpleImputer
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score

import scipy.stats as st

import seaborn as sns

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
experiment = Experiment(api_key="CndJ3YmXyZcxmsV8EccJEuu9C",
                        project_name="NN_Thesis_crossValidation", workspace="paologalligit")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/paologalligit/nn-thesis-crossvalidation/c72c1ca76b564a5dbf9569998912411c



In [3]:
def print_dict(d):
    print("{:<15} {:<15} {:<10}".format('NeuralNetwork','MEAN','STD'))
    for k, v in d.items():
        print("{:<15} {:<15} {:<10}".format(k, round(v['MEAN'], 4), round(v['STD'], 4)))

In [4]:
df = pd.read_csv('fifa19.csv')

In [5]:
df.drop(columns=['Unnamed: 0', 'ID', 'Photo', 'Flag', 'Club Logo', 'Real Face', 'Preferred Foot',
                 'Body Type', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until'],inplace=True)

## Obiettivo: predire valore dei giocatori

Pre-processing: convertire value, wage e release clause da string a float

In [6]:
curs=["Release Clause", "Value", "Wage"]

for cur in curs:
    def curr2val(x):
        x = str(x).replace('€', '')
        if 'K' in x: x = float(str(x).replace('K', '')) * 1000
        else: x = float(str(x).replace('M', '')) * 1000000
        return x
    df[cur] = df[cur].apply(curr2val)
    

Individuare eventuali outlier nella colonna value

In [7]:
def detect_outlier(data, threshold = 3):
    outliers=[]
    mean = np.mean(data)
    std = np.std(data)
    
    for y in data:
        score= (y - mean) / std 
        if np.abs(score) > threshold:
            outliers.append(y)
    return outliers

In [8]:
min_out = min(detect_outlier(df['Value'], threshold = 2))

df = df[df['Value'] < min_out] 
df = df[df['Value'] > 0]

Conversione in interi per le altre label

In [9]:
cols=["LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW","LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM", "LWB", "LDM","CDM", "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB"]
for col in cols:
    df[col] = df[col].str[:-2]
    df[col] = df[col].astype(float)

In [10]:
df['Height'] = df['Height'].str.replace("'",'.')
df['Height'] = df['Height'].astype(float)

df['Weight'] = df['Weight'].str[:-3]
df['Weight'] = df['Weight'].astype(float)

Calcolo correlazione tra i valori per scegliere colonne significative

In [11]:
df_corr = df.corr()

# fig = plt.figure(figsize=(50,20))
# ax = fig.add_subplot(111)
# cax = ax.matshow(df_corr,cmap='coolwarm', vmin=-1, vmax=1)
# fig.colorbar(cax)

# ticks = np.arange(0,len(df_corr.columns),1)
# ax.set_xticks(ticks)
# ax.set_xticklabels(df_corr.columns)
# plt.xticks(rotation=90)
# ax.set_yticks(ticks)
# ax.set_yticklabels(df_corr.columns)

# plt.show()

In [12]:
labels = []
for label in df_corr:
#     if df_corr['Value'][label] < 0 or df_corr['Value'][label] > 0.5: labels.append(label)
    if df_corr['Value'][label] > 0.55: labels.append(label)
        
df_flt = df[labels]        
df_flt.head()      

,Overall,Potential,Value,Wage,LCM,CM,RCM,Reactions,Release Clause
41,88,88,4000000.0,77000.0,NaN,NaN,NaN,79.0,7400000.0
102,85,85,9000000.0,38000.0,70.0,70.0,70.0,85.0,15300000.0
108,85,85,9000000.0,57000.0,63.0,63.0,63.0,83.0,17100000.0
152,84,84,4200000.0,95000.0,63.0,63.0,63.0,80.0,6900000.0
201,83,83,13000000.0,70000.0,NaN,NaN,NaN,78.0,24700000.0


Mescolo le righe del dataset

In [13]:
df_flt = df_flt.sample(frac=1)

train_slice = int(len(df_flt) * 0.8)

train = df_flt[:train_slice]
test = df_flt[train_slice:]

In [14]:
y_train = train.loc[:, ['Value']]
X_train = train.drop(columns='Value')

y_test = test.loc[:, ['Value']]
X_test = test.drop(columns='Value')

Sostiuisco eventuali valori nan con la media della colonna

In [15]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_train)
X_full = imputer.transform(X_train)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y_train)
y_full = imputer.transform(y_train)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_test)
X_test_full = imputer.transform(X_test)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y_test)
y_test_full = imputer.transform(y_test)

Scalo i valori, sia per i caratteri che per il target

In [16]:
# scaler = RobustScaler()
# scaler = scaler.fit(X_train)
# X_train = scaler.transform(X_train)

# X_train_scaled = preprocessing.scale(X_train)
scaler = StandardScaler().fit(X_full)
X_scaled = scaler.transform(X_full)

scaler_train = StandardScaler().fit(y_full)
y_scaled = scaler_train.transform(y_full) 
# X_train_scaled, X_test_scaled

scaler = StandardScaler().fit(X_test_full)
X_test_scaled = scaler.transform(X_test_full)

In [17]:
def coeff_determination(y_test, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_test-y_pred ))
    SS_tot = K.sum(K.square( y_test - K.mean(y_test) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()))

In [18]:
ts_board = TensorBoard(log_dir='value_predictions_v3/{}'.format('kfold_' + str(time())))

In [19]:
def mean_abs_error(prediction, target):
    res = 0
    tot = 0
    for i in range(len(target)):
        if target[i][0] != 0 and not np.isnan(prediction[i][0]):
#             print(prediction[i][0], target[i][0])
            res += abs(prediction[i][0] - target[i][0])
            tot += 1
#     print(tot, len(target), len(prediction))
    return round(res / tot, 2)

# Cross-validation
### K-Fold

In [20]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
kfold = KFold(n_splits=10, random_state=42, shuffle=False)

In [21]:
def build_nn_H1(neurons, n_per_layer):
    model = Sequential()
    # Adding the input layer
    model.add(Dense(n_per_layer, input_dim = neurons, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
#     model.add(LeakyReLU(alpha=0.01))
#     model.add(Dropout(0.1))
    
    # Adding the first hidden layer
#     model.add(Dense(128, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
#     model.add(LeakyReLU(alpha=0.01))
#     model.add(Dropout(0.1))
    
    # Adding the second hidden layer
#     model.add(Dense(64, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
#     model.add(LeakyReLU(alpha=0.01))
    
    # Adding the output layer
    model.add(Dense(1, activation = 'linear'))
    
    return model

In [22]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

nn64 = build_nn_H1(X_scaled.shape[1], 64)
nn128 = build_nn_H1(X_scaled.shape[1], 128)
nn256 = build_nn_H1(X_scaled.shape[1], 256)
nn512 = build_nn_H1(X_scaled.shape[1], 512)
nn1024 = build_nn_H1(X_scaled.shape[1], 1024)
# opt = SGD(lr=0.01, momentum=0.9)
nn64.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn128.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn256.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn512.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn1024.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])

nn_models = [nn64, nn128, nn256, nn512, nn1024]

Instructions for updating:
Colocations handled automatically by placer.


In [23]:
%%time

nn_neurons = 64
nn_dict = {}
nn_counts = 1

for nn in nn_models:
    
    scores = []
    abs_errors = []
    stage = 0
    
    print('#'*60)
    print('NEURAL NETWORK --> ', nn_counts)
    print('#'*60)
    
    nn_counts += 1
    
    for train, test in kfold.split(X_scaled):

        stage += 1

        print('\nstage --> {}\n'.format(stage))

        X_train_round, X_test_round, y_train_round, y_test_round = X_scaled[train], X_scaled[test], y_scaled[train], y_scaled[test]

        nn.fit(X_train_round, y_train_round, batch_size=125, epochs=100, shuffle=True, verbose=0)

        # predict
        res = nn.predict(X_test_round)
        print('R2 of round: ', round(r2_score(y_test_round, res), 4))

        # evaluate
        loss, acc = nn.evaluate(X_train_round, y_train_round, verbose=1)

        reversed_res = scaler_train.inverse_transform(res)
        reversed_label = scaler_train.inverse_transform(y_test_round)
        abs_errors.append(
            {'ACC': acc * 100, 'MAE': mean_abs_error(reversed_res, reversed_label), 'MAX': max(reversed_label), 'MEAN': round(np.mean(reversed_label), 4)}
        )

        print("loss: %.4f, acc: %.4f%%" % (loss, acc*100))
        if acc < 0:
            print('X_test round:\n')
            print(X_test_round)
            print('#'*60)
            print('y_test_round:\n')
            print(y_test_round)
            print('#'*60)

        scores.append(acc * 100)
        
    nn_dict['nn_{}'.format(nn_neurons)] = {'MEAN': np.mean(scores), 'STD': np.std(scores) * 100}
    nn_neurons *= 2 

############################################################
NEURAL NETWORK -->  1
############################################################

stage --> 1

Instructions for updating:
Use tf.cast instead.
R2 of round:  0.9672
12535/12535 [==============================] - 0s 14us/step
loss: 0.0711, acc: 96.0324%

stage --> 2

R2 of round:  0.9802
12535/12535 [==============================] - 0s 11us/step
loss: 0.0610, acc: 96.6972%

stage --> 3

R2 of round:  0.9783
12535/12535 [==============================] - 0s 11us/step
loss: 0.0590, acc: 96.7161%

stage --> 4

R2 of round:  0.9799
12535/12535 [==============================] - 0s 11us/step
loss: 0.0588, acc: 97.1320%

stage --> 5

R2 of round:  0.9799
12535/12535 [==============================] - 0s 10us/step
loss: 0.0574, acc: 97.4592%

stage --> 6

R2 of round:  0.978
12535/12535 [==============================] - 0s 14us/step
loss: 0.0548, acc: 97.5795%

stage --> 7

R2 of round:  0.9713
12535/12535 [=======================

In [24]:
print_dict(nn_dict)

NeuralNetwork   MEAN            STD       
nn_64           97.2981         60.2518   
nn_128          97.8177         41.0398   
nn_256          98.2102         32.1716   
nn_512          98.2164         19.6934   
nn_1024         98.334          17.8023   


In [25]:
nn_neurons = 64
test_eval = dict()

for nn in nn_models:
    res = nn.predict(X_test_scaled)

    # y_test_wrong = y_scaled[:3483]

    scaler_test = StandardScaler().fit(y_test.values)
    res = scaler_test.inverse_transform(res)

    test_eval['t_{}'.format(nn_neurons)] = {'MAE': mean_abs_error(res, y_test.values), 'MAX': max(y_test.values), 'MEAN': round(np.mean(y_test.values), 2), 'R2': round(r2_score(y_test.values, res) * 100, 4)}

    # scaler_test = StandardScaler().fit(y_test)
    # res = scaler_test.inverse_transform(res)
    # y_test_wrong = scaler_test.inverse_transform(y_test_wrong)
    # print(res)

    # mean_abs_error(res, y_test_wrong), max(y_test_wrong), round(np.mean(y_test_wrong), 2), round(r2_score(y_test_wrong, res), 4)
    nn_neurons *= 2

In [26]:
print("{:<15} {:<15} {:<15} {:<15} {:<15}".format('NeuralNetwork', 'MAE', 'MAX', 'MEAN','R2'))
for k, v in test_eval.items():
    print("{:<15} {:<15} {:<15} {:<15} {:<15}".format(k, v['MAE'], v['MAX'][0], v['MEAN'], v['R2']))   

NeuralNetwork   MAE             MAX             MEAN            R2             
t_64            144965.39       13500000.0      1621035.03      97.5739        
t_128           147991.03       13500000.0      1621035.03      97.6184        
t_256           139333.52       13500000.0      1621035.03      97.849         
t_512           150002.13       13500000.0      1621035.03      97.6027        
t_1024          143280.23       13500000.0      1621035.03      97.7151        


RISULTATO: la rete con un solo strato sembra aver ottenuto risultati migliori con 512 neuroni

### Prova con due strati nascosti (H1 - 512)

In [27]:
def build_nn_H2(neurons, n_per_layer):
    model = Sequential()
    # Adding the input layer
    model.add(Dense(512, input_dim = neurons, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    # Adding the first hidden layer
    model.add(Dense(n_per_layer, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    # Adding the output layer
    model.add(Dense(1, activation = 'linear'))
    
    return model

In [28]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

nn64 = build_nn_H2(X_scaled.shape[1], 64)
nn128 = build_nn_H2(X_scaled.shape[1], 128)
nn256 = build_nn_H2(X_scaled.shape[1], 256)
nn512 = build_nn_H2(X_scaled.shape[1], 512)
nn1024 = build_nn_H2(X_scaled.shape[1], 1024)
# opt = SGD(lr=0.01, momentum=0.9)
nn64.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn128.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn256.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn512.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn1024.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])

nn_models = [nn64, nn128, nn256, nn512, nn1024]

In [29]:
%%time

nn_neurons = 64
nn_dict = {}
nn_counts = 1

for nn in nn_models:
    
    scores = []
    abs_errors = []
    stage = 0
    
    print('#'*60)
    print('NEURAL NETWORK --> ', nn_counts)
    print('#'*60)
    
    nn_counts += 1
    
    for train, test in kfold.split(X_scaled):

        stage += 1

        print('\nstage --> {}\n'.format(stage))

        X_train_round, X_test_round, y_train_round, y_test_round = X_scaled[train], X_scaled[test], y_scaled[train], y_scaled[test]

        nn.fit(X_train_round, y_train_round, batch_size=125, epochs=100, shuffle=True, verbose=0)

        # predict
        res = nn.predict(X_test_round)
        print('R2 of round: ', round(r2_score(y_test_round, res), 4))

        # evaluate
        loss, acc = nn.evaluate(X_train_round, y_train_round, verbose=1)

        reversed_res = scaler_train.inverse_transform(res)
        reversed_label = scaler_train.inverse_transform(y_test_round)
        abs_errors.append(
            {'ACC': acc * 100, 'MAE': mean_abs_error(reversed_res, reversed_label), 'MAX': max(reversed_label), 'MEAN': round(np.mean(reversed_label), 4)}
        )

        print("loss: %.4f, acc: %.4f%%" % (loss, acc*100))
        if acc < 0:
            print('X_test round:\n')
            print(X_test_round)
            print('#'*60)
            print('y_test_round:\n')
            print(y_test_round)
            print('#'*60)

        scores.append(acc * 100)
        
    nn_dict['nn_{}'.format(nn_neurons)] = {'MEAN': np.mean(scores), 'STD': np.std(scores) * 100}
    nn_neurons *= 2 

############################################################
NEURAL NETWORK -->  1
############################################################

stage --> 1

R2 of round:  0.9877
12535/12535 [==============================] - 0s 24us/step
loss: 0.0471, acc: 98.1418%

stage --> 2

R2 of round:  0.9869
12535/12535 [==============================] - 0s 19us/step
loss: 0.0400, acc: 98.6657%

stage --> 3

R2 of round:  0.9886
12535/12535 [==============================] - 0s 19us/step
loss: 0.0378, acc: 98.8437%

stage --> 4

R2 of round:  0.9895
12535/12535 [==============================] - 0s 18us/step
loss: 0.0378, acc: 99.0533%

stage --> 5

R2 of round:  0.9872
12535/12535 [==============================] - 0s 17us/step
loss: 0.0349, acc: 99.2445%

stage --> 6

R2 of round:  0.9926
12535/12535 [==============================] - 0s 19us/step
loss: 0.0320, acc: 99.3800%

stage --> 7

R2 of round:  0.9915
12535/12535 [==============================] - 0s 17us/step
loss: 0.0279, acc: 99.4

In [30]:
print_dict(nn_dict)

NeuralNetwork   MEAN            STD       
nn_64           99.1534         45.4775   
nn_128          99.2563         42.6444   
nn_256          99.3028         38.8859   
nn_512          99.3088         39.6924   
nn_1024         99.3341         45.8342   


In [31]:
nn_neurons = 64
test_eval = dict()

for nn in nn_models:
    res = nn.predict(X_test_scaled)

    scaler_test = StandardScaler().fit(y_test.values)
    res = scaler_test.inverse_transform(res)

    test_eval['t_{}'.format(nn_neurons)] = {'MAE': mean_abs_error(res, y_test.values), 'MAX': max(y_test.values), 'MEAN': round(np.mean(y_test.values), 2), 'R2': round(r2_score(y_test.values, res) * 100, 4)}

    nn_neurons *= 2

In [32]:
print("{:<15} {:<15} {:<15} {:<15} {:<15}".format('NeuralNetwork', 'MAE', 'MAX', 'MEAN','R2'))
for k, v in test_eval.items():
    print("{:<15} {:<15} {:<15} {:<15} {:<15}".format(k, v['MAE'], v['MAX'][0], v['MEAN'], v['R2'] ))

NeuralNetwork   MAE             MAX             MEAN            R2             
t_64            136107.72       13500000.0      1621035.03      98.2002        
t_128           135544.17       13500000.0      1621035.03      98.257         
t_256           133655.11       13500000.0      1621035.03      98.2545        
t_512           138729.92       13500000.0      1621035.03      98.0812        
t_1024          136577.55       13500000.0      1621035.03      98.3437        


### Prova con due strati nascosti (H1 - 256)

In [33]:
def build_nn_H2(neurons, n_per_layer):
    model = Sequential()
    # Adding the input layer
    model.add(Dense(256, input_dim = neurons, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    # Adding the first hidden layer
    model.add(Dense(n_per_layer, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    # Adding the output layer
    model.add(Dense(1, activation = 'linear'))
    
    return model

In [34]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

nn64 = build_nn_H2(X_scaled.shape[1], 64)
nn128 = build_nn_H2(X_scaled.shape[1], 128)
nn256 = build_nn_H2(X_scaled.shape[1], 256)
nn512 = build_nn_H2(X_scaled.shape[1], 512)
nn1024 = build_nn_H2(X_scaled.shape[1], 1024)
# opt = SGD(lr=0.01, momentum=0.9)
nn64.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn128.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn256.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn512.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn1024.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])

nn_models = [nn64, nn128, nn256, nn512, nn1024]

In [35]:
%%time

nn_neurons = 64
nn_dict = {}
nn_counts = 1

for nn in nn_models:
    
    scores = []
    abs_errors = []
    stage = 0
    
    print('#'*60)
    print('NEURAL NETWORK --> ', nn_counts)
    print('#'*60)
    
    nn_counts += 1
    
    for train, test in kfold.split(X_scaled):

        stage += 1

        print('\nstage --> {}\n'.format(stage))

        X_train_round, X_test_round, y_train_round, y_test_round = X_scaled[train], X_scaled[test], y_scaled[train], y_scaled[test]

        nn.fit(X_train_round, y_train_round, batch_size=125, epochs=100, shuffle=True, verbose=0)

        # predict
        res = nn.predict(X_test_round)
        print('R2 of round: ', round(r2_score(y_test_round, res), 4))

        # evaluate
        loss, acc = nn.evaluate(X_train_round, y_train_round, verbose=1)

        reversed_res = scaler_train.inverse_transform(res)
        reversed_label = scaler_train.inverse_transform(y_test_round)
        abs_errors.append(
            {'ACC': acc * 100, 'MAE': mean_abs_error(reversed_res, reversed_label), 'MAX': max(reversed_label), 'MEAN': round(np.mean(reversed_label), 4)}
        )

        print("loss: %.4f, acc: %.4f%%" % (loss, acc*100))
        if acc < 0:
            print('X_test round:\n')
            print(X_test_round)
            print('#'*60)
            print('y_test_round:\n')
            print(y_test_round)
            print('#'*60)

        scores.append(acc * 100)
        
    nn_dict['nn_{}'.format(nn_neurons)] = {'MEAN': np.mean(scores), 'STD': np.std(scores) * 100}
    nn_neurons *= 2 

############################################################
NEURAL NETWORK -->  1
############################################################

stage --> 1

R2 of round:  0.9875
12535/12535 [==============================] - 0s 36us/step
loss: 0.0459, acc: 98.2851%

stage --> 2

R2 of round:  0.9871
12535/12535 [==============================] - 0s 18us/step
loss: 0.0451, acc: 98.3314%

stage --> 3

R2 of round:  0.9897
12535/12535 [==============================] - 0s 19us/step
loss: 0.0388, acc: 98.9928%

stage --> 4

R2 of round:  0.9916
12535/12535 [==============================] - 0s 20us/step
loss: 0.0354, acc: 99.2586%

stage --> 5

R2 of round:  0.9877
12535/12535 [==============================] - 0s 17us/step
loss: 0.0348, acc: 99.2624%

stage --> 6

R2 of round:  0.9934
12535/12535 [==============================] - 0s 18us/step
loss: 0.0334, acc: 99.3696%

stage --> 7

R2 of round:  0.9927
12535/12535 [==============================] - 0s 17us/step
loss: 0.0308, acc: 99.3

In [36]:
print_dict(nn_dict)

NeuralNetwork   MEAN            STD       
nn_64           99.127          43.1505   
nn_128          99.2847         40.9494   
nn_256          99.3262         35.4593   
nn_512          99.3024         42.0276   
nn_1024         99.3819         41.2556   


In [37]:
nn_neurons = 64
test_eval = dict()

for nn in nn_models:
    res = nn.predict(X_test_scaled)

    scaler_test = StandardScaler().fit(y_test.values)
    res = scaler_test.inverse_transform(res)

    test_eval['t_{}'.format(nn_neurons)] = {'MAE': mean_abs_error(res, y_test.values), 'MAX': max(y_test.values), 'MEAN': round(np.mean(y_test.values), 2), 'R2': round(r2_score(y_test.values, res) * 100, 4)}

    nn_neurons *= 2

In [38]:
print("{:<15} {:<15} {:<15} {:<15} {:<15}".format('NeuralNetwork', 'MAE', 'MAX', 'MEAN','R2'))
for k, v in test_eval.items():
    print("{:<15} {:<15} {:<15} {:<15} {:<15}".format(k, v['MAE'], v['MAX'][0], v['MEAN'], v['R2'] ))

NeuralNetwork   MAE             MAX             MEAN            R2             
t_64            134708.36       13500000.0      1621035.03      98.1627        
t_128           136847.86       13500000.0      1621035.03      98.1763        
t_256           137330.89       13500000.0      1621035.03      98.198         
t_512           141966.77       13500000.0      1621035.03      98.1409        
t_1024          143702.25       13500000.0      1621035.03      97.9681        


### Prova con due strati nascosti (H1 - 128)

In [39]:
def build_nn_H2(neurons, n_per_layer):
    model = Sequential()
    # Adding the input layer
    model.add(Dense(128, input_dim = neurons, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    # Adding the first hidden layer
    model.add(Dense(n_per_layer, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    # Adding the output layer
    model.add(Dense(1, activation = 'linear'))
    
    return model

In [40]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

nn64 = build_nn_H2(X_scaled.shape[1], 64)
nn128 = build_nn_H2(X_scaled.shape[1], 128)
nn256 = build_nn_H2(X_scaled.shape[1], 256)
nn512 = build_nn_H2(X_scaled.shape[1], 512)
nn1024 = build_nn_H2(X_scaled.shape[1], 1024)
# opt = SGD(lr=0.01, momentum=0.9)
nn64.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn128.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn256.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn512.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn1024.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])

nn_models = [nn64, nn128, nn256, nn512, nn1024]

In [41]:
%%time

nn_neurons = 64
nn_dict = {}
nn_counts = 1

for nn in nn_models:
    
    scores = []
    abs_errors = []
    stage = 0
    
    print('#'*60)
    print('NEURAL NETWORK --> ', nn_counts)
    print('#'*60)
    
    nn_counts += 1
    
    for train, test in kfold.split(X_scaled):

        stage += 1

        print('\nstage --> {}\n'.format(stage))

        X_train_round, X_test_round, y_train_round, y_test_round = X_scaled[train], X_scaled[test], y_scaled[train], y_scaled[test]

        nn.fit(X_train_round, y_train_round, batch_size=125, epochs=100, shuffle=True, verbose=0)

        # predict
        res = nn.predict(X_test_round)
        print('R2 of round: ', round(r2_score(y_test_round, res), 4))

        # evaluate
        loss, acc = nn.evaluate(X_train_round, y_train_round, verbose=1)

        reversed_res = scaler_train.inverse_transform(res)
        reversed_label = scaler_train.inverse_transform(y_test_round)
        abs_errors.append(
            {'ACC': acc * 100, 'MAE': mean_abs_error(reversed_res, reversed_label), 'MAX': max(reversed_label), 'MEAN': round(np.mean(reversed_label), 4)}
        )

        print("loss: %.4f, acc: %.4f%%" % (loss, acc*100))
        if acc < 0:
            print('X_test round:\n')
            print(X_test_round)
            print('#'*60)
            print('y_test_round:\n')
            print(y_test_round)
            print('#'*60)

        scores.append(acc * 100)
        
    nn_dict['nn_{}'.format(nn_neurons)] = {'MEAN': np.mean(scores), 'STD': np.std(scores) * 100}
    nn_neurons *= 2 

############################################################
NEURAL NETWORK -->  1
############################################################

stage --> 1

R2 of round:  0.9868
12535/12535 [==============================] - 0s 38us/step
loss: 0.0489, acc: 98.0550%

stage --> 2

R2 of round:  0.9897
12535/12535 [==============================] - 0s 19us/step
loss: 0.0443, acc: 98.5115%

stage --> 3

R2 of round:  0.9877
12535/12535 [==============================] - 0s 22us/step
loss: 0.0403, acc: 98.6333%

stage --> 4

R2 of round:  0.9889
12535/12535 [==============================] - 0s 21us/step
loss: 0.0392, acc: 98.7825%

stage --> 5

R2 of round:  0.9828
12535/12535 [==============================] - 0s 18us/step
loss: 0.0368, acc: 98.8563%

stage --> 6

R2 of round:  0.992
12535/12535 [==============================] - 0s 18us/step
loss: 0.0380, acc: 98.9705%

stage --> 7

R2 of round:  0.9855
12535/12535 [==============================] - 0s 19us/step
loss: 0.0353, acc: 99.15

In [42]:
print_dict(nn_dict)

NeuralNetwork   MEAN            STD       
nn_64           98.8841         38.6008   
nn_128          99.1262         38.2752   
nn_256          99.2226         38.6135   
nn_512          99.3168         42.3036   
nn_1024         99.3229         41.8602   


In [43]:
nn_neurons = 64
test_eval = dict()

for nn in nn_models:
    res = nn.predict(X_test_scaled)

    scaler_test = StandardScaler().fit(y_test.values)
    res = scaler_test.inverse_transform(res)

    test_eval['t_{}'.format(nn_neurons)] = {'MAE': mean_abs_error(res, y_test.values), 'MAX': max(y_test.values), 'MEAN': round(np.mean(y_test.values), 2), 'R2': round(r2_score(y_test.values, res) * 100, 4)}

    nn_neurons *= 2

In [44]:
print("{:<15} {:<15} {:<15} {:<15} {:<15}".format('NeuralNetwork', 'MAE', 'MAX', 'MEAN','R2'))
for k, v in test_eval.items():
    print("{:<15} {:<15} {:<15} {:<15} {:<15}".format(k, v['MAE'], v['MAX'][0], v['MEAN'], v['R2'] ))

NeuralNetwork   MAE             MAX             MEAN            R2             
t_64            138641.74       13500000.0      1621035.03      98.2178        
t_128           134634.01       13500000.0      1621035.03      98.2689        
t_256           145657.85       13500000.0      1621035.03      98.0459        
t_512           143588.99       13500000.0      1621035.03      97.8723        
t_1024          144390.69       13500000.0      1621035.03      97.9785        


### Prova con tre strati nascosti (H1 - 512, H2 - 64)

In [45]:
def build_nn_H3(neurons, n_per_layer):
    model = Sequential()
    # Adding the input layer
    model.add(Dense(512, input_dim = neurons, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    # Adding the first hidden layer
    model.add(Dense(64, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    
    model.add(Dense(n_per_layer, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    # Adding the output layer
    model.add(Dense(1, activation = 'linear'))
    
    return model

In [46]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

nn64 = build_nn_H3(X_scaled.shape[1], 64)
nn128 = build_nn_H3(X_scaled.shape[1], 128)
nn256 = build_nn_H3(X_scaled.shape[1], 256)
nn512 = build_nn_H3(X_scaled.shape[1], 512)
nn1024 = build_nn_H3(X_scaled.shape[1], 1024)
# opt = SGD(lr=0.01, momentum=0.9)
nn64.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn128.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn256.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn512.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn1024.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])

nn_models = [nn64, nn128, nn256, nn512, nn1024]

In [47]:
%%time

nn_neurons = 64
nn_dict = {}
nn_counts = 1

for nn in nn_models:
    
    scores = []
    abs_errors = []
    stage = 0
    
    print('#'*60)
    print('NEURAL NETWORK --> ', nn_counts)
    print('#'*60)
    
    nn_counts += 1
    
    for train, test in kfold.split(X_scaled):

        stage += 1

        print('\nstage --> {}\n'.format(stage))

        X_train_round, X_test_round, y_train_round, y_test_round = X_scaled[train], X_scaled[test], y_scaled[train], y_scaled[test]

        nn.fit(X_train_round, y_train_round, batch_size=125, epochs=100, shuffle=True, verbose=0)

        # predict
        res = nn.predict(X_test_round)
        print('R2 of round: ', round(r2_score(y_test_round, res), 4))

        # evaluate
        loss, acc = nn.evaluate(X_train_round, y_train_round, verbose=1)

        reversed_res = scaler_train.inverse_transform(res)
        reversed_label = scaler_train.inverse_transform(y_test_round)
        abs_errors.append(
            {'ACC': acc * 100, 'MAE': mean_abs_error(reversed_res, reversed_label), 'MAX': max(reversed_label), 'MEAN': round(np.mean(reversed_label), 4)}
        )

        print("loss: %.4f, acc: %.4f%%" % (loss, acc*100))
        if acc < 0:
            print('X_test round:\n')
            print(X_test_round)
            print('#'*60)
            print('y_test_round:\n')
            print(y_test_round)
            print('#'*60)

        scores.append(acc * 100)
        
    nn_dict['nn_{}'.format(nn_neurons)] = {'MEAN': np.mean(scores), 'STD': np.std(scores) * 100}
    nn_neurons *= 2 

############################################################
NEURAL NETWORK -->  1
############################################################

stage --> 1

R2 of round:  0.988
12535/12535 [==============================] - 1s 59us/step
loss: 0.0439, acc: 98.3213%

stage --> 2

R2 of round:  0.9881
12535/12535 [==============================] - 0s 29us/step
loss: 0.0397, acc: 98.9256%

stage --> 3

R2 of round:  0.9895
12535/12535 [==============================] - 0s 35us/step
loss: 0.0341, acc: 99.2632%

stage --> 4

R2 of round:  0.9908
12535/12535 [==============================] - 0s 26us/step
loss: 0.0310, acc: 99.4187%

stage --> 5

R2 of round:  0.9879
12535/12535 [==============================] - 0s 31us/step
loss: 0.0326, acc: 99.4148%

stage --> 6

R2 of round:  0.9943
12535/12535 [==============================] - 0s 32us/step
loss: 0.0296, acc: 99.4815%

stage --> 7

R2 of round:  0.9935
12535/12535 [==============================] - 0s 28us/step
loss: 0.0250, acc: 99.60

In [48]:
print_dict(nn_dict)

NeuralNetwork   MEAN            STD       
nn_64           99.3389         40.1488   
nn_128          99.2334         50.935    
nn_256          99.2712         34.6346   
nn_512          99.2122         30.2396   
nn_1024         99.1989         41.1612   


In [49]:
nn_neurons = 64
test_eval = dict()

for nn in nn_models:
    res = nn.predict(X_test_scaled)

    scaler_test = StandardScaler().fit(y_test.values)
    res = scaler_test.inverse_transform(res)

    test_eval['t_{}'.format(nn_neurons)] = {'MAE': mean_abs_error(res, y_test.values), 'MAX': max(y_test.values), 'MEAN': round(np.mean(y_test.values), 2), 'R2': round(r2_score(y_test.values, res) * 100, 4)}

    nn_neurons *= 2

In [50]:
print("{:<15} {:<15} {:<15} {:<15} {:<15}".format('NeuralNetwork', 'MAE', 'MAX', 'MEAN','R2'))
for k, v in test_eval.items():
    print("{:<15} {:<15} {:<15} {:<15} {:<15}".format(k, v['MAE'], v['MAX'][0], v['MEAN'], v['R2'] ))

NeuralNetwork   MAE             MAX             MEAN            R2             
t_64            139719.77       13500000.0      1621035.03      98.0239        
t_128           134200.32       13500000.0      1621035.03      98.2085        
t_256           132223.74       13500000.0      1621035.03      98.2737        
t_512           132525.49       13500000.0      1621035.03      98.3477        
t_1024          137201.69       13500000.0      1621035.03      98.2118        


### Prova con tre strati nascosti (H1 - 256, H2 - 128)

In [51]:
def build_nn_H3(neurons, n_per_layer):
    model = Sequential()
    # Adding the input layer
    model.add(Dense(256, input_dim = neurons, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    # Adding the first hidden layer
    model.add(Dense(128, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    
    model.add(Dense(n_per_layer, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    # Adding the output layer
    model.add(Dense(1, activation = 'linear'))
    
    return model

In [52]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

nn64 = build_nn_H3(X_scaled.shape[1], 64)
nn128 = build_nn_H3(X_scaled.shape[1], 128)
nn256 = build_nn_H3(X_scaled.shape[1], 256)
nn512 = build_nn_H3(X_scaled.shape[1], 512)
nn1024 = build_nn_H3(X_scaled.shape[1], 1024)
# opt = SGD(lr=0.01, momentum=0.9)
nn64.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn128.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn256.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn512.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])
nn1024.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])

nn_models = [nn64, nn128, nn256, nn512, nn1024]

In [53]:
%%time

nn_neurons = 64
nn_dict = {}
nn_counts = 1

for nn in nn_models:
    
    scores = []
    abs_errors = []
    stage = 0
    
    print('#'*60)
    print('NEURAL NETWORK --> ', nn_counts)
    print('#'*60)
    
    nn_counts += 1
    
    for train, test in kfold.split(X_scaled):

        stage += 1

        print('\nstage --> {}\n'.format(stage))

        X_train_round, X_test_round, y_train_round, y_test_round = X_scaled[train], X_scaled[test], y_scaled[train], y_scaled[test]

        nn.fit(X_train_round, y_train_round, batch_size=125, epochs=100, shuffle=True, verbose=0)

        # predict
        res = nn.predict(X_test_round)
        print('R2 of round: ', round(r2_score(y_test_round, res), 4))

        # evaluate
        loss, acc = nn.evaluate(X_train_round, y_train_round, verbose=1)

        reversed_res = scaler_train.inverse_transform(res)
        reversed_label = scaler_train.inverse_transform(y_test_round)
        abs_errors.append(
            {'ACC': acc * 100, 'MAE': mean_abs_error(reversed_res, reversed_label), 'MAX': max(reversed_label), 'MEAN': round(np.mean(reversed_label), 4)}
        )

        print("loss: %.4f, acc: %.4f%%" % (loss, acc*100))
        if acc < 0:
            print('X_test round:\n')
            print(X_test_round)
            print('#'*60)
            print('y_test_round:\n')
            print(y_test_round)
            print('#'*60)

        scores.append(acc * 100)
        
    nn_dict['nn_{}'.format(nn_neurons)] = {'MEAN': np.mean(scores), 'STD': np.std(scores) * 100}
    nn_neurons *= 2 

############################################################
NEURAL NETWORK -->  1
############################################################

stage --> 1

R2 of round:  0.9863
12535/12535 [==============================] - 1s 66us/step
loss: 0.0428, acc: 98.6423%

stage --> 2

R2 of round:  0.9878
12535/12535 [==============================] - 0s 30us/step
loss: 0.0365, acc: 99.1050%

stage --> 3

R2 of round:  0.9904
12535/12535 [==============================] - 0s 32us/step
loss: 0.0303, acc: 99.3931%

stage --> 4

R2 of round:  0.9896
12535/12535 [==============================] - 0s 31us/step
loss: 0.0278, acc: 99.5129%

stage --> 5

R2 of round:  0.9918
12535/12535 [==============================] - 0s 30us/step
loss: 0.0271, acc: 99.5775%

stage --> 6

R2 of round:  0.9958
12535/12535 [==============================] - 0s 30us/step
loss: 0.0238, acc: 99.6639%

stage --> 7

R2 of round:  0.9931
12535/12535 [==============================] - 0s 37us/step
loss: 0.0249, acc: 99.6

In [54]:
print_dict(nn_dict)

NeuralNetwork   MEAN            STD       
nn_64           99.4807         34.0051   
nn_128          99.4154         32.6256   
nn_256          99.4201         23.8407   
nn_512          99.3731         35.3707   
nn_1024         99.4043         29.6436   


In [55]:
nn_neurons = 64
test_eval = dict()

for nn in nn_models:
    res = nn.predict(X_test_scaled)

    scaler_test = StandardScaler().fit(y_test.values)
    res = scaler_test.inverse_transform(res)

    test_eval['t_{}'.format(nn_neurons)] = {'MAE': mean_abs_error(res, y_test.values), 'MAX': max(y_test.values), 'MEAN': round(np.mean(y_test.values), 2), 'R2': round(r2_score(y_test.values, res) * 100, 4)}

    nn_neurons *= 2

In [56]:
print("{:<15} {:<15} {:<15} {:<15} {:<15}".format('NeuralNetwork', 'MAE', 'MAX', 'MEAN','R2'))
for k, v in test_eval.items():
    print("{:<15} {:<15} {:<15} {:<15} {:<15}".format(k, v['MAE'], v['MAX'][0], v['MEAN'], v['R2'] ))

NeuralNetwork   MAE             MAX             MEAN            R2             
t_64            138617.52       13500000.0      1621035.03      98.1044        
t_128           136560.85       13500000.0      1621035.03      98.1742        
t_256           135910.14       13500000.0      1621035.03      98.149         
t_512           135312.39       13500000.0      1621035.03      98.3084        
t_1024          142477.57       13500000.0      1621035.03      97.8538        
